In [3]:
import pandas as pd
import numpy as np
from DS_healpix_fragmentation import radec2pix
from DS_Planck_Unet import val_pix

In [4]:
files = {'pz14' : '/home/rt2122/Data/detected_cats/full_pz14_thr0.1_step8.csv', 
        'pz40' : '/home/rt2122/Data/detected_cats/full_pz40_thr0.1_step8.csv',
        'pz_act10' : '/home/rt2122/Data/detected_cats/full_pz_act10_thr0.1_step8.csv',
        'pz_act14' : '/home/rt2122/Data/detected_cats/full_pz_act20_thr0.1_step8.csv',
        'pz_act_rot27' : '/home/rt2122/Data/detected_cats/act_ros_rot/val_pz_act_rot_ep27_thr0.1_step8.csv',
        'pz_act_drop0.3_ep14' : 
             '/home/rt2122/Data/detected_cats/act_ros_drop0.3/pz_act_drop0.3_ep14_thr0.1_step8.csv',
        'pz_act_drop0.4_ep25' : 
             '/home/rt2122/Data/detected_cats/act_ros_drop0.4/pz_act_drop0.4_ep25_thr0.1_step8.csv',
        'pz_act_norm2' : '/home/rt2122/Data/detected_cats/act_ros_norm/pz_act_norm2_thr0.1_step8.csv'}

In [5]:
for model in files:
    df = pd.read_csv(files[model])
    df['catalog'][df['catalog'] == 'act_erosita'] = 'actwp'
    df['catalog'][df['catalog'] == 'act'] = 'actwp'
    df.to_csv(files[model], index=False)

<ipython-input-5-45f549c6dac2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['catalog'][df['catalog'] == 'act_erosita'] = 'actwp'
<ipython-input-5-45f549c6dac2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['catalog'][df['catalog'] == 'act'] = 'actwp'


In [6]:
res_df = []
for model in files:
    df = pd.read_csv(files[model])
    df = df.iloc[np.in1d(radec2pix(df['RA'], df['DEC'], 2), val_pix)]
    df.index = np.arange(len(df))
    line = {'name' : model}
    for cat in ['planck_z', 'planck_no_z', 'mcxcwp', 'actwp']:
        cur_df = df[df['catalog'] == cat]
        cur_df.index = np.arange(len(cur_df))
        line[cat] = np.count_nonzero(cur_df['status'] == 'tp') / len(cur_df)
    line['fp'] = np.count_nonzero(df['status'] == 'fp')
    res_df.append(pd.DataFrame(line, index=[0]))
res_df = pd.concat(res_df, ignore_index=True)
res_df

,name,planck_z,planck_no_z,mcxcwp,actwp,fp
0,pz14,0.931818,0.821429,0.155340,0.120983,1055
1,pz40,0.931818,0.785714,0.145631,0.130435,1180
2,pz_act10,0.901515,0.821429,0.097087,0.089202,624
3,pz_act14,0.825758,0.750000,0.058252,0.043478,557
4,pz_act_rot27,0.901515,0.750000,0.135922,0.066163,351
5,pz_act_drop0.3_ep14,0.901515,0.785714,0.077670,0.079395,526
6,pz_act_drop0.4_ep25,0.893939,0.642857,0.087379,0.062382,510
7,pz_act_norm2,0.863636,0.500000,0.058252,0.054820,424


In [7]:
res_df.to_csv('/home/rt2122/Data/models_stat/by_split_cats.csv', index=False)

In [14]:
def cut_cat(df, dict_cut = {'z' : [-np.inf, np.inf], 'M500' : [-np.inf, np.inf], 'b' :[-np.inf, np.inf]}, 
           big_pix=None):
    from astropy.coordinates import SkyCoord
    from astropy import units as u
 
    sc = SkyCoord(ra=np.array(df['RA'])*u.degree, 
                  dec=np.array(df['DEC'])*u.degree, frame='icrs')
    df['b'] = sc.galactic.b.degree
    for prm in dict_cut:
        if prm == 'b':
            df = df[np.abs(df[prm]) >= dict_cut[prm][0]]
            df = df[np.abs(df[prm]) < dict_cut[prm][1]]
            df.index = np.arange(len(df))
        else: 
            df = df[df[prm] >= dict_cut[prm][0]]
            df = df[df[prm] < dict_cut[prm][1]]
            df.index = np.arange(len(df))
    
    if not (big_pix is None):
        pix2 = radec2pix(df['RA'], df['DEC'], 2)
        df = df[np.in1d(pix2, big_pix)]
        df.index = np.arange(len(df))
    
    return df

In [18]:
def gen_tables(det_cats_dict, true_cats_dir, dict_cut = {'z' : [-np.inf, np.inf], 'M500' : [-np.inf, np.inf], 
                        'b' :[-np.inf, np.inf]}, big_pix = None, match_dist=5/60, n_try=200):
    import os
    from astropy.coordinates import SkyCoord
    from astropy import units as u
    from DS_data_transformation import calc_error
    
    true_cats_files = next(os.walk(true_cats_dir))[-1]
    true_cats_files = [os.path.join(true_cats_dir, file) for file in true_cats_files]
    
    true_cats = {os.path.splitext(os.path.basename(file))[0] : pd.read_csv(file) for file in true_cats_files}
    det_cats = {name : 
                pd.read_csv(det_cats_dict[name]) for name in det_cats_dict}
    
    comp_df = []
    recall_df = []
    
    for name in det_cats:
        df = det_cats[name]
        df = df[df['status'] != 'fn']
        df.index = np.arange(len(df))
        if 'b' in dict_cut:
            det_cats[name] = cut_cat(df, {'b' : dict_cut['b']}, big_pix)
        else:
            det_cats[name] = cut_cat(df, {}, big_pix)
    for name in true_cats:
        true_cats[name] = cut_cat(true_cats[name], dict_cut, big_pix)
    
    for det_name in det_cats:
        det = det_cats[det_name]
        line = {}
        line_r = {}

        det_sc = SkyCoord(ra=np.array(det['RA'])*u.degree, 
                      dec=np.array(det['DEC'])*u.degree, frame='icrs') 

        for tr_name in true_cats: 
            tr = true_cats[tr_name]
            tr_sc = SkyCoord(ra=np.array(tr['RA'])*u.degree, 
                          dec=np.array(tr['DEC'])*u.degree, frame='icrs')
            
            idx, d2d, _ = tr_sc.match_to_catalog_sky(det_sc)
            matched = d2d.degree <= match_dist
            
            line[tr_name] = np.count_nonzero(matched)
            line[tr_name+'_err'], line[tr_name+'_std'] = calc_error(det, tr, n_try=n_try)

            line_r[tr_name] = line[tr_name] / len(tr)
            
        line['all'] = len(det)
        line['fp'] = np.count_nonzero(det['status'] == 'fp')
        line_r['fp'] = line['fp']
        line_r['all'] = line['all']
        comp_df.append(pd.DataFrame(line, index=[det_name]))
        recall_df.append(pd.DataFrame(line_r, index=[det_name]))
        line = {}
    
    for tr_name in true_cats: 
        line[tr_name] = len(true_cats[tr_name])
        line[tr_name+'_err'] = 0
    line['fp'] = 0
    line['all'] = 0
    comp_df.append(pd.DataFrame(line, index=['all']))
    
    comp_df = pd.concat(comp_df)
    recall_df = pd.concat(recall_df)
    return comp_df, recall_df

In [19]:
comp_df, recall_df = gen_tables(files, '/home/rt2122/Data/original_catalogs/csv', big_pix=val_pix, dict_cut={})

<ipython-input-14-fea80ac82c83>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['b'] = sc.galactic.b.degree


In [21]:
recall_df

,PSZ2,MCXC,RM,ACT,fp,all
pz14,0.92500,0.433735,0.045764,0.210963,1055,1281
pz40,0.91875,0.421687,0.048361,0.219269,1180,1409
pz_act10,0.90000,0.391566,0.035054,0.159468,624,814
pz_act14,0.83125,0.355422,0.029211,0.121262,557,716
pz_act_rot27,0.88125,0.415663,0.029536,0.157807,351,540
pz_act_drop0.3_ep14,0.89375,0.361446,0.032782,0.166113,526,717
pz_act_drop0.4_ep25,0.86250,0.379518,0.031483,0.149502,510,688
pz_act_norm2,0.81250,0.361446,0.024667,0.139535,424,587


In [22]:
recall_df.to_csv('/home/rt2122/Data/models_stat/by_orig_cats.csv', index=False)